In [16]:
# Import the necessary libraries
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier  
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt



In [10]:
df=pd.read_csv('AMA Demand Supply from DW .csv')
df=df[['YEAR','SECTOR','PARENT WATER TYPE OR SECTOR','QUANTITY']]
df

,YEAR,SECTOR,PARENT WATER TYPE OR SECTOR,QUANTITY
0,1985,Municipal,Municipal,308110
1,1985,Municipal,Municipal,152321
2,1985,Municipal,Municipal,31656
3,1985,Municipal,Municipal,9177
4,1985,Municipal,Municipal,128889
...,...,...,...,...
7648,2020,Agricultural,Effluent,230
7649,2020,Agricultural,Other,44
7650,2020,Indian,Indian,18349
7651,2020,Indian,Groundwater,262


In [11]:
data=df.loc[df['SECTOR']=='Municipal']
data

,YEAR,SECTOR,PARENT WATER TYPE OR SECTOR,QUANTITY
0,1985,Municipal,Municipal,308110
1,1985,Municipal,Municipal,152321
2,1985,Municipal,Municipal,31656
3,1985,Municipal,Municipal,9177
4,1985,Municipal,Municipal,128889
...,...,...,...,...
7620,2020,Municipal,Groundwater,6136
7621,2020,Municipal,CAP,0
7622,2020,Municipal,Effluent,4655
7623,2020,Municipal,CAP,114285


In [12]:
# data['PARENT WATER TYPE OR SECTOR'].value_counts()
data=data.loc[(data['PARENT WATER TYPE OR SECTOR']=='Effluent')|(data['PARENT WATER TYPE OR SECTOR']=='Groundwater')|(data['PARENT WATER TYPE OR SECTOR']=='CAP')|(data['PARENT WATER TYPE OR SECTOR']=='Surface Water')]
data=data.rename(columns = {'PARENT WATER TYPE OR SECTOR':'Source'}).reset_index()
data=data[['YEAR','Source','QUANTITY']] 
data

,YEAR,Source,QUANTITY
0,1985,Groundwater,225094
1,1985,Surface Water,360702
2,1985,Groundwater,13159
3,1985,Groundwater,4579
4,1985,Surface Water,210
...,...,...,...
755,2020,Groundwater,6136
756,2020,CAP,0
757,2020,Effluent,4655
758,2020,CAP,114285


In [13]:
table = pd.pivot_table(data, values='QUANTITY', index=['YEAR', 'Source'], aggfunc=np.sum)
table

QUANTITY
YEAR Source                 
1985 Groundwater      360052
     Surface Water    360912
1986 CAP               13036
     Effluent           3426
     Groundwater      434856
...                      ...
2019 Surface Water    309907
2020 CAP              506361
     Effluent         172277
     Groundwater      384010
     Surface Water    346823

[142 rows x 1 columns]

In [14]:
data = pd.DataFrame(table.to_records())
data

,YEAR,Source,QUANTITY
0,1985,Groundwater,360052
1,1985,Surface Water,360912
2,1986,CAP,13036
3,1986,Effluent,3426
4,1986,Groundwater,434856
...,...,...,...
137,2019,Surface Water,309907
138,2020,CAP,506361
139,2020,Effluent,172277
140,2020,Groundwater,384010


In [18]:
# One hot encode the 'Source' column
encoder = OneHotEncoder()
source_encoded = encoder.fit_transform(data[['Source']])
source_encoded_df = pd.DataFrame(source_encoded.toarray(), columns=encoder.get_feature_names(['Source']))
source_encoded_df


,Source_CAP,Source_Effluent,Source_Groundwater,Source_Surface Water
0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
137,0.0,0.0,0.0,1.0
138,1.0,0.0,0.0,0.0
139,0.0,1.0,0.0,0.0
140,0.0,0.0,1.0,0.0


In [19]:
data = pd.concat([data, source_encoded_df], axis=1)
data

,YEAR,Source,QUANTITY,Source_CAP,Source_Effluent,Source_Groundwater,Source_Surface Water
0,1985,Groundwater,360052,0.0,0.0,1.0,0.0
1,1985,Surface Water,360912,0.0,0.0,0.0,1.0
2,1986,CAP,13036,1.0,0.0,0.0,0.0
3,1986,Effluent,3426,0.0,1.0,0.0,0.0
4,1986,Groundwater,434856,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
137,2019,Surface Water,309907,0.0,0.0,0.0,1.0
138,2020,CAP,506361,1.0,0.0,0.0,0.0
139,2020,Effluent,172277,0.0,1.0,0.0,0.0
140,2020,Groundwater,384010,0.0,0.0,1.0,0.0


In [20]:
# Create a list of independent variables
X = data[['YEAR', 'Source_CAP', 'Source_Effluent', 'Source_Groundwater', 'Source_Surface Water']]

# Create a list of the dependent variable
y = data['QUANTITY']


In [21]:
# Create a Linear Regression model
model = LinearRegression()

# Create a DecisionTreeClassifier model
#model=DecisionTreeClassifier(criterion='entropy', random_state=0)  

# Fit the model to the data
model.fit(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
year = int(input("Enter a year for prediction: "))


# Make the predictions for all sources
predictions = model.predict([[year, 1, 0, 0, 0], [year, 0, 1, 0, 0], [year, 0, 0, 1, 0], [year, 0, 0, 0, 1]])

# Print the predictions
print(f'Predicted usage for Source_CAP in {year}: {predictions[0]}')
print(f'Predicted usage for Source_Effluent in {year}: {predictions[1]}')
print(f'Predicted usage for Source_Groundwater in {year}: {predictions[2]}')
print(f'Predicted usage for Source_Surface Water in {year}: {predictions[3]}')

# Overall usages
all_usages=predictions[0]+predictions[1]+predictions[2]+predictions[3]
print(f'Total usages of  {year} is : {all_usages}')

Enter a year for prediction: 2020
Predicted usage for Source_CAP in 2020: 399325.16093225963
Predicted usage for Source_Effluent in 2020: 134149.50378940254
Predicted usage for Source_Groundwater in 2020: 430860.14769170526
Predicted usage for Source_Surface Water in 2020: 444684.0088028163
Total usages of  2020 is : 1409018.8212161837
